# Task 2 多元数值向量的假设检验
关于多元数值向量
> 单变量问题和多变量问题的区别是什么——主要在于微观与宏观的区别。单变量问题研究的是具体某一个指标的性质，而多变量问题则是将多个指标作为一个整体，用它们去描述一个对象的性质。

## 1 均值向量的检验
> 均值向量检验的内容架构与均值检验的架构是一样的，主要内容有：
> - 比较一组数据的总体均值向量与一个固定向量是否相等的检验
> - 比较两组数据的总体均值向量之间是否相等的检验
> - 比较两组以上的多组数据的总体均值向量之间是否相等的检验          

> 在多元领域，有一个检验和t检验非常类似——Hotelling T2检验，该检验在变量个数等于1时就退化成了t检验。

<img src='pic/Task2_1.png'>

## 2 从多元到一元——寻找显著的变量

> 多元均值向量的检验可以告诉我们不同分析对象的某个宏观性质（如城市企业的发展状况）是否存在显著区别，如果我们拒绝了原假设，便可以说明它们确实存在显著区别。但是在实际分析中，仅仅得出“它们存在显著区别”的结论是远远不够的，我们会想知道，究竟是什么变量导致了这种显著区别呢？这个时候，我们就要是用一元的均值向量检验来回答这一问题。

> 也就是说，在实际分析当中，往往都是始于“多元”，终于“一元”。

<img src='pic/Task2_2.png'>

## 作业
为研究东、中、西部各省市规模以上的企业发展状况，我们收集了各城市企业的主要经济指标，包括：总资产贡献率、资产负债率、流动资产周转次数、工业成本费用利润率、产品销售率。我们用变量“类别”定义了各类城市，其中1为东部城市；2为中部城市；3为西部城市。数据文件为homework2.xlsx。假设显著性水平为，问：

1. 对三个类别的城市进行均值向量间的两两比较，查看结果

2. 对三个类别的城市同时进行均值向量间的比较，查看结果

3. 承接问题2，你认为哪些变量导致了三个类别城市均值向量的差异？说出你的理由。

In [1]:
# 加载必要的包
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.stats import f
from itertools import product

In [2]:
#加载作业数据
df = pd.read_excel('data/homework2.xlsx',index_col = 0)

In [3]:
df.head()

,类别,总资产贡献率,资产负债率,流动资产周转次数,工业成本费用利润率,产品销售率
地区,,,,,,
北京,1,7.56,51.85,1.50,7.77,99.04
天津,1,17.41,63.47,2.26,9.41,98.89
河北,1,13.83,59.40,3.27,6.19,97.79
山西,2,9.54,69.61,1.71,5.71,97.34
内蒙古,3,14.90,61.25,2.29,12.12,97.10


In [4]:
#对三个类别城市进行聚合
group = df.groupby(by = "类别").groups
len(group)

3

In [5]:
col = df.columns.tolist()[1:]
df1 = df.loc[group[1]][col]
df2 = df.loc[group[2]][col]
df3 = df.loc[group[3]][col]

In [6]:
df1.head()

,总资产贡献率,资产负债率,流动资产周转次数,工业成本费用利润率,产品销售率
地区,,,,,
北京,7.56,51.85,1.50,7.77,99.04
天津,17.41,63.47,2.26,9.41,98.89
河北,13.83,59.40,3.27,6.19,97.79
辽宁,14.45,57.93,3.09,5.37,97.79
上海,12.61,50.62,1.93,6.61,98.94


### 1 对三个类别的城市进行均值向量间的两两比较，查看结果
使用组间均值相等性检验

In [7]:
def multi_unparied_data(group1:pd.DataFrame,group2:pd.DataFrame,confidence=0.05):
    # 计算检验统计量
    n1=len(group1)
    n2=len(group2)
    p=np.shape(group1)[1] # 变量维度
    mean1=np.mean(group1).values.T
    mean2=np.mean(group2).values.T
    S1=np.cov(group1.T)
    S2=np.cov(group2.T)
    Sp=((n1-1)*S1+(n2-1)*S2)/(n1+n2-2)
    T2=n1*n2*(mean1-mean2).T@np.linalg.inv(Sp)@(mean1-mean2)/(n1+n2)
    Test_statistics=(n1+n2-p-1)*T2/(p*(n1+n2-2))

    # 计算p值
    pvalue=f.sf(Test_statistics,p,n1+n2-p-1)

    # 比较p值与显著性水平
    if pvalue<confidence:
        print('在显著性水平{0:}下，两组样本所在总体的均值向量不相等。(p={1:.4f})'.format(confidence,pvalue))
    else:
        print('在显著性水平{0:}下，两组样本所在总体的均值向量相等。(p={1:.4f})'.format(confidence,pvalue))
    return pvalue

In [8]:
dflis = [df1,df2,df3]
lis = list(range(len(dflis)))
name = ["东部城市","中部城市","西部城市"]
for source, target in product(lis,lis):
    if source >= target:
        continue
    print('----比较{}和{}的均值----'.format(name[source],name[target]))
    multi_unparied_data(dflis[source],dflis[target])
        

----比较东部城市和中部城市的均值----
在显著性水平0.05下，两组样本所在总体的均值向量相等。(p=0.2793)
----比较东部城市和西部城市的均值----
在显著性水平0.05下，两组样本所在总体的均值向量不相等。(p=0.0097)
----比较中部城市和西部城市的均值----
在显著性水平0.05下，两组样本所在总体的均值向量不相等。(p=0.0470)


可以看出在显著型水平0.05下，东部和中部城市均值相等，他们和西部城市的均值都不相等。

### 2 对三个类别的城市同时进行均值向量间的比较，查看结果
样本组数是3，使用多元方差分析

In [9]:
from statsmodels.multivariate.manova import MANOVA

In [10]:
model=MANOVA.from_formula("+".join(col)+'~ 类别', data=df).mv_test()

In [11]:
print(model.results['类别']['stat'])

                           Value Num DF Den DF  F Value      Pr > F
Wilks' lambda           0.535029      5     25  4.34528  0.00554155
Pillai's trace          0.464971      5     25  4.34528  0.00554155
Hotelling-Lawley trace  0.869057      5     25  4.34528  0.00554155
Roy's greatest root     0.869057      5     25  4.34528  0.00554155


在上述报告表中，看Wilks' lambda的p值。此处$p=0.00554155$，我们不能接受原假设，3组均值向量有显著差别。

### 3 承接问题2，你认为哪些变量导致了三个类别城市均值向量的差异？说出你的理由。

则需要对变量进行分别分析

In [12]:
def single_val_foneway(df1,df2,df3,col,confidence = 0.05):
    for val in col:
        print('----对 {} 进行一元方差分析----'.format(val))
        fre = stats.f_oneway(df1[val].values,df2[val].values,df3[val].values)
        print(fre)
        pvalue = fre.pvalue
        if pvalue<confidence:
            print('在显著性水平{0:}下，三组样本所在总体的均值不相等。(p={1:.4f})'.format(confidence,pvalue))
        else:
            print('在显著性水平{0:}下，三组样本所在总体的均值相等。(p={1:.4f})'.format(confidence,pvalue))

In [13]:
single_val_foneway(df1,df2,df3,col)

----对 总资产贡献率 进行一元方差分析----
F_onewayResult(statistic=3.5776324463321805, pvalue=0.04133778417613981)
在显著性水平0.05下，三组样本所在总体的均值不相等。(p=0.0413)
----对 资产负债率 进行一元方差分析----
F_onewayResult(statistic=0.7703999845168195, pvalue=0.47239023585585066)
在显著性水平0.05下，三组样本所在总体的均值相等。(p=0.4724)
----对 流动资产周转次数 进行一元方差分析----
F_onewayResult(statistic=6.484356111904944, pvalue=0.0048515449323179)
在显著性水平0.05下，三组样本所在总体的均值不相等。(p=0.0049)
----对 工业成本费用利润率 进行一元方差分析----
F_onewayResult(statistic=2.125754135219324, pvalue=0.13820126688080314)
在显著性水平0.05下，三组样本所在总体的均值相等。(p=0.1382)
----对 产品销售率 进行一元方差分析----
F_onewayResult(statistic=4.847034218142545, pvalue=0.015573643638731122)
在显著性水平0.05下，三组样本所在总体的均值不相等。(p=0.0156)


由于 流动资产周转次数 p值很小，让我们剔除它，再做一次多元方差分析

In [14]:
col.remove("流动资产周转次数")

In [15]:
model=MANOVA.from_formula("+".join(col)+'~ 类别', data=df).mv_test()
print(model.results['类别']['stat'])

                           Value Num DF Den DF  F Value      Pr > F
Wilks' lambda           0.551704      4     26  5.28168  0.00299547
Pillai's trace          0.448296      4     26  5.28168  0.00299547
Hotelling-Lawley trace  0.812566      4     26  5.28168  0.00299547
Roy's greatest root     0.812566      4     26  5.28168  0.00299547


在上述报告表中，看Wilks' lambda的p值。此处$p=0.00299547$，我们依然不能接受原假设，3组均值向量有显著差别。

由于 产品销售率 p值也很小，让我们再剔除它，再做一次多元方差分析

In [16]:
col.remove("产品销售率")

In [17]:
model=MANOVA.from_formula("+".join(col)+'~ 类别', data=df).mv_test()
print(model.results['类别']['stat'])

                           Value Num DF Den DF  F Value     Pr > F
Wilks' lambda           0.671812      3     27  4.39661  0.0121144
Pillai's trace          0.328188      3     27  4.39661  0.0121144
Hotelling-Lawley trace  0.488513      3     27  4.39661  0.0121144
Roy's greatest root     0.488513      3     27  4.39661  0.0121144


在上述报告表中，看Wilks' lambda的p值。此处$p=0.0121144$，我们依然不能接受原假设，3组均值向量有显著差别。

由于 总资产贡献率 p值也很小，让我们继续剔除它，再做一次多元方差分析

In [18]:
col.remove("总资产贡献率")

In [19]:
model=MANOVA.from_formula("+".join(col)+'~ 类别', data=df).mv_test()
print(model.results['类别']['stat'])

                           Value Num DF Den DF  F Value     Pr > F
Wilks' lambda           0.735822      2     28  5.02634  0.0136402
Pillai's trace          0.264178      2     28  5.02634  0.0136402
Hotelling-Lawley trace  0.359024      2     28  5.02634  0.0136402
Roy's greatest root     0.359024      2     28  5.02634  0.0136402


在上述报告表中，看Wilks' lambda的p值。此处$p=0.0136402$，我们依然不能接受原假设，3组均值向量有显著差别。

此时再次剔除变量已经没有意义，因为只剩下两个变量

In [20]:
col

['资产负债率', '工业成本费用利润率']

我们可以认为，所有的变量共同导致了三个类别城市均值向量的差异，因为无论剔除哪个变量，3组均值向量均有显著区别。尽管'资产负债率', '工业成本费用利润率'在单变量方差分析中是显著的，但是多元方差分析中却不显著。